In [1]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from Method import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data= pd.read_csv("D:\Study\DSEB 63 - NEU\Year 3\Sem 2\Thesis\Reviews.csv")
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [3]:
# Clean text
df_sampled = preprocess_data(data)
df = df_sampled[['Text']]
tokenized_reviews = df['Text'].apply(preprocess_text)
tokenized_reviews

0        [love, anna, ginger, thin, cooky, hoping, find...
1        [dont, chocolate, bell, plantation, site, four...
2        [first, coffee, tried, keurig, disappointed, f...
3        [item, shipped, unsealed, plastic, within, bro...
4        [cat, picky, eater, sometime, found, weruvia, ...
                               ...                        
49995    [mother, love, product, heard, great, thing, a...
49996    [compared, favorite, breakfast, kashi, honey, ...
49997    [love, good, coffee, bought, keurig, lavazza, ...
49998    [bought, month, buying, popcorn, flavorful, de...
49999    [popchips, original, potato, chip, deelish, be...
Name: Text, Length: 50000, dtype: object

In [4]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [5]:
# Creating the object for LsA model using gensim library
LSA = gensim.models.LsiModel
# Build LsA model
lsa_model = LSA(corpus=doc_term_matrix, id2word=dictionary, num_topics=10,chunksize=1000)

In [ ]:
lsa_model.print_topics()

[(0,
  '0.372*"like" + 0.284*"taste" + 0.247*"coffee" + 0.221*"food" + 0.214*"flavor" + 0.209*"product" + 0.196*"good" + 0.188*"would" + 0.136*"water" + 0.126*"dont"'),
 (1,
  '-0.778*"coffee" + 0.439*"food" + 0.130*"product" + -0.109*"taste" + 0.099*"cat" + -0.097*"flavor" + 0.087*"ingredient" + 0.072*"eating" + 0.070*"chicken" + 0.066*"dog"'),
 (2,
  '-0.576*"food" + -0.487*"coffee" + 0.336*"taste" + 0.261*"like" + 0.175*"flavor" + 0.162*"water" + -0.133*"cat" + 0.086*"chocolate" + 0.078*"drink" + 0.073*"sugar"'),
 (3,
  '0.766*"water" + -0.327*"like" + 0.230*"product" + 0.122*"bottled" + 0.103*"essentia" + 0.098*"drinking" + -0.097*"flavor" + -0.097*"taste" + 0.096*"bottle" + -0.095*"chocolate"'),
 (4,
  '0.697*"product" + -0.333*"water" + -0.289*"like" + -0.277*"food" + -0.194*"taste" + 0.149*"amazon" + 0.099*"would" + 0.081*"price" + 0.076*"hair" + 0.071*"order"'),
 (5,
  '0.502*"like" + -0.452*"flavor" + 0.260*"product" + -0.189*"good" + 0.157*"food" + -0.143*"eating" + -0.130*"l

In [ ]:
# print('\nPerplexity: ', lsa_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. the lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lsa = CoherenceModel(model=lsa_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lsa = coherence_model_lsa.get_coherence()
print('\nCoherence Score: ', coherence_lsa)


Coherence Score:  0.4093188715093509


***
Method to find optimal number of topics
***

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, end, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, end, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, end=50, step=1)

KeyboardInterrupt: 

In [ ]:
# Show graph
end=50; start=2; step=1;
x = range(start, end, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()# Print the coherence scores

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[7]
model_topics = optimal_model.show_topics(formatted=False)
optimal_model.print_topics(num_words=10)